In [1]:
import os
# os.environ['APP_HOST'] = 'http://192.168.60.25:8005'
os.environ['USER_ID'] = 'abbd86d4-d3c0-48d4-be5a-a6cefd22002b'
os.environ['PROJECT_ID'] = '9d9bca30-a314-4f93-b9c2-f368a0b0135b'
os.environ['TENANT_ID'] = 'cb7d3189-7b2d-4d72-951f-a70a79931f52'

import sys
sys.version
sys.path.append('/Users/phaneendra/IdeaProjects/razor-sdk')

In [2]:
def generator_function():
    print("in generator")
    for i in range(10):
        print("yielding")
        yield [1], [2]

from razor import Pipeline, inputs, outputs, Block
@outputs.atomic.generic('data')
class DataGen(Block):
    def run(self, data):
        def data_gen():
            import tensorflow as tf
            print('in generator')
            dataset = tf.data.Dataset.from_generator(generator_function, output_types=tuple([tf.float32 for i in range(2)]))
            print('created data gen')
            return list(dataset)
        data.put(data_gen())




Couldn't setup host configurations. You may not be running this in the platform's Jupyter
Failed to load configuration file. Using default configs


In [3]:
from razor.core.blocks import ContainerExecutor
data_gen= DataGen().executor(ContainerExecutor(cores=4, memory=4096, gpu=1))


#data_gen.execute() -> This works


from razor.core.blocks.executors import SubProcessExecutor

pipeline = Pipeline(targets=[data_gen])


In [4]:
from razor.platform import engines
deployed = engines('Engine-1-dev1').execute(pipeline) # Change `Engine-1-dev1` to the engine name in your tenant 

Exception: {"timestamp":"2020-05-28T11:57:25.417+0000","status":405,"error":"Method Not Allowed","message":"Request method 'POST' not supported","path":"/rztai/engine-manager/engine/un-auth/list"}

In [ ]:
deployed.monitor()